# Simulaciones de Fermentaciones

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Cargar datos finales
df = pd.read_csv("sin_outliers_rounded.csv")
df = df[df["Tiempo de fermentación"] <= 24]

# Crear listas para guardar las series simuladas
series_simuladas = []

# Tiempo en pasos de 1 minutos
T_total = 24  # duración máxima en horas
dt = 1/60
tiempo = np.arange(0, T_total + dt, dt)

# Simulación por fila
for idx, fila in df.iterrows():
    # Datos base
    variedad = fila["Variedad"]
    altura = fila["Altura"]
    cantidad = fila["Cantidad (L)"]
    temp_final = fila["Temperatura"]
    ph_final = fila["pH"]
    t_fermentacion = min(fila["Tiempo de fermentación"], 24)

    # Recortar tiempo a duración real
    tiempo_real = tiempo[tiempo <= t_fermentacion]

    # === Temperatura ===
    T_ini = 25 + 0.002 * altura + 0.5 * np.random.rand()
    k_temp = 0.2 + 0.05 * np.random.rand()
    temp = T_ini + (temp_final - T_ini) * (1 - np.exp(-k_temp * tiempo_real))
    temp += np.random.normal(0, 0.01, len(temp))

    # === pH ===
    ph_ini = 6.5 - 0.001 * altura + 0.05 * np.random.rand()
    delta_ph = ph_ini - ph_final
    k_ph = 0.4 + 0.05 * np.random.rand()
    ph = ph_ini - delta_ph * np.log1p(k_ph * tiempo_real) / np.log1p(k_ph * t_fermentacion)
    ph += np.random.normal(0, 0.02, len(ph))

    # === Presión (airlock con posibles obstrucciones) ===
    P_base = 1.0  # atm (presión atmosférica)
    delta_P = 0.05 + 0.05 * np.random.rand()  # leve sobrepresión
    t_peak = t_fermentacion / 2
    alpha = 1.5 / (t_fermentacion ** 2)

    # Obstrucción parcial del airlock (5% de probabilidad)
    if np.random.rand() < 0.05:
        delta_P *= 3
        alpha /= 3

    presion = P_base + delta_P * np.exp(-alpha * (tiempo_real - t_peak) ** 2)
    presion += np.random.normal(0, 0.005, len(presion))  # ruido leve

    # === Alcohol (sigmoide más lenta) ===
    A_max = 2.0 + 0.3 * np.random.rand()
    k_alc = 0.5 + 0.2 * np.random.rand()
    t0_alc = t_fermentacion / 3
    alcohol = A_max / (1 + np.exp(-k_alc * (tiempo_real - t0_alc)))
    alcohol += np.random.normal(0, 0.05, len(alcohol))

    # Guardar en DataFrame
    df_temp = pd.DataFrame({
        "Tiempo (h)": tiempo_real,
        "Temperatura": temp,
        "pH": ph,
        "Presión": presion,
        "Alcohol": alcohol,
        "Variedad": variedad,
        "Altura": altura,
        "Cantidad (L)": cantidad,
        "Indice": idx
    })

    series_simuladas.append(df_temp)

# Unir todas las series
df_simulado = pd.concat(series_simuladas, ignore_index=True)

# Guardar en CSV
df_simulado.to_csv("series_simuladas_1m.csv", index=False)

print("✅ Simulación completada y guardada.")

## Visualización Rápida

In [ ]:
# Graficar una fermentación simulada
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
sns.set()

df_simulado = pd.read_csv("series_simuladas_1m.csv")

id_mostrar = 1001  # Índice de fermentación
df_ejemplo = df_simulado[df_simulado["Indice"] == id_mostrar]

fig, axs = plt.subplots(2, 2, figsize=(12, 8))
axs[0, 0].plot(df_ejemplo["Tiempo (h)"], df_ejemplo["Temperatura"])
axs[0, 0].set_title("Temperatura")

axs[0, 1].plot(df_ejemplo["Tiempo (h)"], df_ejemplo["pH"])
axs[0, 1].set_title("pH")

axs[1, 0].plot(df_ejemplo["Tiempo (h)"], df_ejemplo["Presión"])
axs[1, 0].set_title("Presión")

axs[1, 1].plot(df_ejemplo["Tiempo (h)"], df_ejemplo["Alcohol"])
axs[1, 1].set_title("Alcohol en el aire")

plt.tight_layout()
plt.show()

# Modelo con pasos de 1 min

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import joblib

import onnx
import skl2onnx
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

# --------------------------
# Configuración
# --------------------------
ventana_historial = 10    # minutos de historial usados como entrada
horizonte_prediccion = 30 # minutos hacia el futuro que queremos predecir
frecuencia_datos = 1    # minutos entre muestras en la simulación

# --------------------------
# Cargar datos simulados
# --------------------------
df = pd.read_csv("series_simuladas_1m.csv")

# Calcular pasos
pasos_historial = int(ventana_historial / frecuencia_datos)
pasos_a_predecir = int(horizonte_prediccion / frecuencia_datos)

# Crear variable objetivo: temperatura futura
df['temp_futura'] = df['Temperatura'].shift(-pasos_a_predecir)

# Crear características a partir del historial para TODAS las variables
variables = ["Temperatura", "pH", "Presión", "Altura", "Cantidad (L)", "Variedad"]
for var in variables:
    for i in range(1, pasos_historial + 1):
        df[f"{var}_t-{i}"] = df[var].shift(i)

# Eliminar filas con NaN generados por el shift
df = df.dropna()

# --------------------------
# Entrenamiento del modelo
# --------------------------
X = df[[f"{var}_t-{i}" for var in variables for i in range(1, pasos_historial + 1)]]
y = df['temp_futura']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

modelo = GradientBoostingRegressor(n_estimators=300, learning_rate=0.05, random_state=42)
modelo.fit(X_train, y_train)

# --------------------------
# Evaluación
# --------------------------
y_pred = modelo.predict(X_test)

mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"MAE:  {mae:.3f} °C")
print(f"RMSE: {rmse:.3f} °C")
print(f"R²:   {r2:.3f}")

# ======================
# Exportar a ONNX
# ======================
# Definir forma de entrada (número de features)
initial_type = [('float_input', FloatTensorType([None, X_train.shape[1]]))]

onnx_model = convert_sklearn(modelo, initial_types=initial_type, target_opset=8)
onnx.save_model(onnx_model, "modelo_Temp1m.onnx")